# Python Class Final Project
## Program Description:
### Twitter analysis
The program has 2 modes of operation. The "Live" mode works on incoming real-time tweets on the given tag while the search mode searches the previous tweets related to  the tag. We use tweepy package to get the twitter data.

The Live mode uses streaming mode while the Search mode uses the api method of the tweepy package. The data gathered by either of the calls is then displayed to the web page using flask package. The stream method being blocking on data, we use threads to unblock the main program and display live partial and streaming data to the web.

For the sentiment analysis, we use the textBlob package which is a natural language analysis package.

Foe streaming display on the web, we have used the http-equiv mata-data to refresh the page automatically after a few seconds. Thus fetching the latest data collected from the stream. We do not have to wait to collect all the data and hold the page from displaying.


## File Structure:
### root directory:
get_tweets.py: This is the main and single file.
    - process_tweets - This is the function that processes all the collected tweets including removing non-alphanumaric characters, removing urls etc and passing the text to textBlob. Getting back and storing the sentiment score and red or green highlight.
    - get_twitter - This authenticates the app with the tweeter, checks the global state and request state. Based on the same it calls the API->search method or launches a seprate thread for collecting the streaming data. The immediately available data is processed and readied for display on the web
    - twitter_form - This is the function which creates the interface between flask web display and the get_twitter functions. It gets the request variables, processes them if needed and also renders the final web display.
 
 
### templates directory:
my_twitter.html: This is the single page form and display table formatted templete for both, 
                 taking user request and also displaying the twitter feeds.
 
### output file:
my_twitter.txt:  While the output is sent to the web, a local copy of the tweets and the score 
                 is also created in the current dir. This file is over-written each time the 
                 web-page is refreshed and is always in sync with the data being displayed.

In [ ]:
#my_twitter.html:

<html>
    <head>
        <title>My Twitter</title>
        <meta http-equiv="refresh" content="{{refresh_time}}">
    </head>
    <body>
        <form>
          <p>
          Method: <select name="method" value={{method}}>
                     <option value=Live>Live</option>
                     <option value=Search
                        {% if method %}
                                selected=true
                        {%endif%}
                        >Search</option>
                  </select>
          Refresh Rate: <input type="text" name="refresh_time" value={{refresh_time}}>
          </p>
          <p>Max Tweets:
          <input type="text" name="max_tweets" value={{max_tweets}}>
          </p>
          <p>Hash Tag:
          <input type="text" name="tweet_tag" value={{tweet_tag}}>
          </p>
          <input type="submit" value="Submit">
        </form>
         <table border="1" width=90%>
        {% for tweet in tweets %}
            <tr
                {% if tweet.green %}
                    bgcolor=green
                {%endif%}
                {% if tweet.red %}
                    bgcolor=red
                {%endif%} >
            <td>{{tweet.text}}</td>
            <td>{{tweet.score}}</td>
            </tr>
        {% endfor %}
         </table>
    </body>
</html>


In [ ]:
#get_tweets.py

#!/usr/bin/env python

import re
import tweepy
import json
from textblob import TextBlob
from thread import start_new_thread
from flask import Flask
from flask import render_template
from flask import request


#Twitter API credentials
consumer_key = ''
consumer_secret = ''
access_key = ''
access_secret = ''
output_file = "my_tweets.txt"
tweets = []
tweet_index = 0
max_tweets = 10
tweet_tag = "#royalwedding"
refresh_time = 100000
method = 2 # 1 - Live 2 - search
old_method = 2
stream=None

def process_tweets():
    file = open(output_file, 'w')
    for i in range(0, max_tweets):
        tweet = tweets[tweet_index - 1 - i].text
        if tweet == "" :
            i -= 1;
            break
        text_only_tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

        #Write to file. Add new line
        file.write(text_only_tweet)
        file.write("\n")
        analysis = TextBlob(text_only_tweet)
        tweets[tweet_index - 1 - i].score = analysis.sentiment.polarity
        if tweets[tweet_index - 1 - i].score >= 0.4:
            tweets[tweet_index - 1 - i].green = 1
        else:
            tweets[tweet_index - 1 - i].green = 0
        if tweets[tweet_index - 1 - i].score <= -0.4:
            tweets[tweet_index - 1 - i].red = 1
        else:
            tweets[tweet_index - 1 - i].red = 0
        file.write("Analysis score is %1.8f\n" % analysis.sentiment.polarity)
    file.close()
    return i+1

def stream_twitter():
    global stream
    global auth
    listen = StdOutListener()
    stream = tweepy.Stream(auth, listen, async=True, tweet_mode='extended')
    stream.filter(track=[tweet_tag])

def get_twitter():
    global tweet_tag
    global tweets
    global max_tweets
    global method
    global old_method
    global tweet_index
    global stream
    global auth
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)

    if method == 1:
        old_method = 1
        if not stream :
            start_new_thread(stream_twitter,())
        n = process_tweets()
    elif method == 2:
        if old_method == 1 :
            old_method = 2
            stream.disconnect()
            stream=None
        api = tweepy.API(auth)
        fetched_tweets = api.search(q = tweet_tag, count = max_tweets, tweet_mode='extended')
        tweet_index = len(fetched_tweets) - 1
        for i in range(tweet_index,-1,-1):
            tweets[tweet_index - i].text = fetched_tweets[i].full_text
        n = process_tweets()

class StdOutListener(tweepy.StreamListener):
    def on_data(self, data):
        if method != 1:
            return True
        global tweet_index
        global tweets

        decode_data = json.loads(data)
        tweets[tweet_index].text = decode_data["text"]
        tweet_index += 1
        tweet_index %= max_tweets
        return True

    def on_error(self, status):
        print status

class tweet:
    def __init__(self):
        self.text = ""
        self.score = 0
        self.red = 0
        self.green = 0

app = Flask(__name__)

@app.route("/", methods=['GET', 'POST'])
def twitter_form():
    global tweet_tag
    global tweets
    global max_tweets
    global refresh_time
    global method
    global tweet_index

    new_max_tweets = request.args.get("max_tweets")
    if new_max_tweets :
        max_tweets = int(new_max_tweets)
        tweet_index %= max_tweets

    new_refresh_time =request.args.get('refresh_time')
    if new_refresh_time :
        refresh_time = int(new_refresh_time)

    get_method =request.args.get('method')
    if get_method == "Live":
        method = 1
        if refresh_time == 100000:
            refresh_time = 10
        if refresh_time > 30:
            refresh_time = 30
    else:
        method = 2
        refresh_time = 100000

    new_tweet_tag = request.args.get("tweet_tag")
    if new_tweet_tag :
        tweet_tag = new_tweet_tag
    for i in range(len(tweets),max_tweets,-1):
        tweets.pop()
    for i in range(len(tweets),max_tweets):
        tweets.append(tweet())
    get_twitter()
    new_tweets = []
    for i in range(0,max_tweets):
        new_tweets.append(tweets[(i + tweet_index)%max_tweets])
    return render_template("my_twitter.html",
                           max_tweets=max_tweets,
                           method=(method != 1),
                           refresh_time=refresh_time,
                           tweet_tag=tweet_tag,
                           tweets=new_tweets)

if __name__ == "__main__":
    app.run(host ='0.0.0.0',port=80, debug=True)

